In [1]:
# notebook上で可視化した図を表示するためのコマンド
%matplotlib inline

# ライラブリをimport
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# ファイルを読み込み
df_gm1cluA = pd.read_table('dihda-gm1clu.dat', delim_whitespace=True, names=['frame', 'A_psi', 'A_phi'])
df_gm1cluA = df_gm1cluA[df_gm1cluA['frame'] != '#Frame'].astype(float).drop('frame', axis=1)

df_gm1cluB = pd.read_table('dihdb-gm1clu.dat', delim_whitespace=True, names=['frame', 'B_psi', 'B_phi'])
df_gm1cluB = df_gm1cluB[df_gm1cluB['frame'] != '#Frame'].astype(float).drop('frame', axis=1)

df_gm1cluC = pd.read_table('dihdc-gm1clu.dat', delim_whitespace=True, names=['frame', 'C_psi', 'C_phi'])
df_gm1cluC = df_gm1cluC[df_gm1cluC['frame'] != '#Frame'].astype(float).drop('frame', axis=1)

df_gm1cluD = pd.read_table('dihdd-gm1clu.dat', delim_whitespace=True, names=['frame', 'D_psi', 'D_phi'])
df_gm1cluD = df_gm1cluD[df_gm1cluD['frame'] != '#Frame'].astype(float).drop('frame', axis=1)

# state0
df_gm1cluABCD = pd.concat([df_gm1cluA, df_gm1cluB, df_gm1cluC, df_gm1cluD], axis=1)
df_gm1cluS0 = df_gm1cluABCD[(df_gm1cluABCD['A_psi'] < 100) & (df_gm1cluABCD['A_psi'] > -100)]
df_gm1cluS0 = df_gm1cluS0[(df_gm1cluS0['A_phi'] > -100) & (df_gm1cluS0['A_phi'] < 100)]

#df_gm1cluS0 = df_gm1cluS0[(df_gm1cluS0['C_psi'] < 100) & (df_gm1cluS0['C_psi'] > -100)]
#df_gm1cluS0 = df_gm1cluS0[(df_gm1cluS0['C_phi'] > -100) & (df_gm1cluS0['C_phi'] < 110)]

df_gm1cluS0 = df_gm1cluS0[(df_gm1cluS0['D_psi'] < -125) | (df_gm1cluS0['D_psi'] > 100)]
df_gm1cluS0 = df_gm1cluS0[(df_gm1cluS0['D_phi'] > -100) & (df_gm1cluS0['D_phi'] < 100)]

# state1
df_gm1cluABCD = pd.concat([df_gm1cluA, df_gm1cluB, df_gm1cluC, df_gm1cluD], axis=1)
df_gm1cluS1 = df_gm1cluABCD[(df_gm1cluABCD['A_psi'] < 100) & (df_gm1cluABCD['A_psi'] > -100)]
df_gm1cluS1 = df_gm1cluS1[(df_gm1cluS1['A_phi'] > -100) & (df_gm1cluS1['A_phi'] < 100)]

#df_gm1cluS1 = df_gm1cluS1[(df_gm1cluS1['C_psi'] < 100) & (df_gm1cluS1['C_psi'] > -100)]
#df_gm1cluS1 = df_gm1cluS1[(df_gm1cluS1['C_phi'] > -100) & (df_gm1cluS1['C_phi'] < 110)]

df_gm1cluS1 = df_gm1cluS1[(df_gm1cluS1['D_psi'] > -125) & (df_gm1cluS1['D_psi'] < 100)]
df_gm1cluS1 = df_gm1cluS1[(df_gm1cluS1['D_phi'] > -100) & (df_gm1cluS1['D_phi'] < 100)]

# state2
df_gm1cluABCD = pd.concat([df_gm1cluA, df_gm1cluB, df_gm1cluC, df_gm1cluD], axis=1)
df_gm1cluS2 = df_gm1cluABCD[(df_gm1cluABCD['A_psi'] < 100) & (df_gm1cluABCD['A_psi'] > -100)]
df_gm1cluS2 = df_gm1cluS2[(df_gm1cluS2['A_phi'] < -100) | (df_gm1cluS2['A_phi'] > 100)]

#df_gm1cluS2 = df_gm1cluS2[(df_gm1cluS2['C_psi'] < 100) & (df_gm1cluS2['C_psi'] > -100)]
#df_gm1cluS2 = df_gm1cluS2[(df_gm1cluS2['C_phi'] > -100) & (df_gm1cluS2['C_phi'] < 110)]

df_gm1cluS2 = df_gm1cluS2[(df_gm1cluS2['D_psi'] < -125) | (df_gm1cluS2['D_psi'] > 100)]
df_gm1cluS2 = df_gm1cluS2[(df_gm1cluS2['D_phi'] > -100) & (df_gm1cluS2['D_phi'] < 100)]

# state3
df_gm1cluABCD = pd.concat([df_gm1cluA, df_gm1cluB, df_gm1cluC, df_gm1cluD], axis=1)
df_gm1cluS3 = df_gm1cluABCD[(df_gm1cluABCD['A_psi'] < 100) & (df_gm1cluABCD['A_psi'] > -100)]
df_gm1cluS3 = df_gm1cluS3[(df_gm1cluS3['A_phi'] < -100) | (df_gm1cluS3['A_phi'] > 100)]

#df_gm1cluS3 = df_gm1cluS3[(df_gm1cluS3['C_psi'] < 100) & (df_gm1cluS3['C_psi'] > -100)]
#df_gm1cluS3 = df_gm1cluS3[(df_gm1cluS3['C_phi'] > -100) & (df_gm1cluS3['C_phi'] < 110)]

df_gm1cluS3 = df_gm1cluS3[(df_gm1cluS3['D_psi'] > -125) & (df_gm1cluS3['D_psi'] < 100)]
df_gm1cluS3 = df_gm1cluS3[(df_gm1cluS3['D_phi'] > -100) & (df_gm1cluS3['D_phi'] < 100)]

# state4
df_gm1cluABCD = pd.concat([df_gm1cluA, df_gm1cluB, df_gm1cluC, df_gm1cluD], axis=1)
df_gm1cluS4 = df_gm1cluABCD[(df_gm1cluABCD['A_psi'] > 100) | (df_gm1cluABCD['A_psi'] < -100)]
df_gm1cluS4 = df_gm1cluS4[(df_gm1cluS4['A_phi'] > -100) & (df_gm1cluS4['A_phi'] < 100)]

#df_gm1cluS4 = df_gm1cluS4[(df_gm1cluS4['C_psi'] < 100) & (df_gm1cluS4['C_psi'] > -100)]
#df_gm1cluS4 = df_gm1cluS4[(df_gm1cluS4['C_phi'] > -100) & (df_gm1cluS4['C_phi'] < 110)]

df_gm1cluS4 = df_gm1cluS4[(df_gm1cluS4['D_psi'] < -125) | (df_gm1cluS4['D_psi'] > 100)]
df_gm1cluS4 = df_gm1cluS4[(df_gm1cluS4['D_phi'] > -100) & (df_gm1cluS4['D_phi'] < 100)]

# state5
df_gm1cluABCD = pd.concat([df_gm1cluA, df_gm1cluB, df_gm1cluC, df_gm1cluD], axis=1)
df_gm1cluS5 = df_gm1cluABCD[(df_gm1cluABCD['A_psi'] > 100) | (df_gm1cluABCD['A_psi'] < -100)]
df_gm1cluS5 = df_gm1cluS5[(df_gm1cluS5['A_phi'] > -100) & (df_gm1cluS5['A_phi'] < 100)]

#df_gm1cluS5 = df_gm1cluS5[(df_gm1cluS5['C_psi'] < 100) & (df_gm1cluS5['C_psi'] > -100)]
#df_gm1cluS5 = df_gm1cluS5[(df_gm1cluS5['C_phi'] > -100) & (df_gm1cluS5['C_phi'] < 110)]

df_gm1cluS5 = df_gm1cluS5[(df_gm1cluS5['D_psi'] > -125) & (df_gm1cluS5['D_psi'] < 100)]
df_gm1cluS5 = df_gm1cluS5[(df_gm1cluS5['D_phi'] > -100) & (df_gm1cluS5['D_phi'] < 100)]

# state6
df_gm1cluABCD = pd.concat([df_gm1cluA, df_gm1cluB, df_gm1cluC, df_gm1cluD], axis=1)
df_gm1cluS6 = df_gm1cluABCD[(df_gm1cluABCD['A_psi'] < 100) & (df_gm1cluABCD['A_psi'] > -100)]
df_gm1cluS6 = df_gm1cluS6[(df_gm1cluS6['A_phi'] > -100) & (df_gm1cluS6['A_phi'] < 100)]

#df_gm1cluS6 = df_gm1cluS6[(df_gm1cluS6['C_psi'] < 100) & (df_gm1cluS6['C_psi'] > -100)]
#df_gm1cluS6 = df_gm1cluS6[(df_gm1cluS6['C_phi'] < -100) | (df_gm1cluS6['C_phi'] > 110)]

df_gm1cluS6 = df_gm1cluS6[(df_gm1cluS6['D_psi'] < -125) | (df_gm1cluS6['D_psi'] > 100)]
df_gm1cluS6 = df_gm1cluS6[(df_gm1cluS6['D_phi'] > -100) & (df_gm1cluS6['D_phi'] < 100)]

# state7
df_gm1cluABCD = pd.concat([df_gm1cluA, df_gm1cluB, df_gm1cluC, df_gm1cluD], axis=1)
df_gm1cluS7 = df_gm1cluABCD[(df_gm1cluABCD['A_psi'] < 100) & (df_gm1cluABCD['A_psi'] > -100)]
df_gm1cluS7 = df_gm1cluS7[(df_gm1cluS7['A_phi'] > -100) & (df_gm1cluS7['A_phi'] < 100)]

df_gm1cluS7 = df_gm1cluS7[(df_gm1cluS7['C_psi'] > 100) | (df_gm1cluS7['C_psi'] < -100)]
df_gm1cluS7 = df_gm1cluS7[(df_gm1cluS7['C_phi'] > -100) & (df_gm1cluS7['C_phi'] < 110)]

df_gm1cluS7 = df_gm1cluS7[(df_gm1cluS7['D_psi'] < -125) | (df_gm1cluS7['D_psi'] > 100)]
df_gm1cluS7 = df_gm1cluS7[(df_gm1cluS7['D_phi'] > -100) & (df_gm1cluS7['D_phi'] < 100)]

In [3]:
df_gm1cluABCD.head(10)

,A_psi,A_phi,B_psi,B_phi,C_psi,C_phi,D_psi,D_phi
1,33.2267,-23.4482,37.3582,12.1085,36.6327,13.3237,-66.8286,-5.2195
2,31.4266,-4.6571,23.0820,21.5680,49.0754,-24.2250,-78.5554,5.5114
3,52.8997,-20.4379,34.0867,11.7559,47.4844,9.1815,-74.9093,12.0130
4,60.7078,-5.8933,27.5400,10.8030,35.8645,-27.4155,-83.0305,7.1414
5,47.1127,-4.7791,45.6054,5.7287,47.2786,-30.9148,-60.9972,0.9490
6,30.2567,1.9422,22.9949,22.2816,49.3375,-31.2979,-57.1619,-13.0349
7,40.7122,17.9262,28.1716,15.5641,51.1263,-16.8958,-62.2357,2.9399
8,54.5460,14.8525,38.2042,20.0267,38.2367,28.1904,-69.0351,-16.9519
9,44.7372,15.7248,37.3429,9.4237,34.5563,46.6959,-67.0581,7.6733
10,30.7839,16.6240,41.7213,-24.0618,33.6298,4.6506,-57.2612,3.1490


In [4]:
# 角度をsin, cosで変換
psi_sin = []
psi_cos = []
phi_sin = []
phi_cos = []

for i, v in df_gm1cluA.iterrows():
    psi_sin.append(np.sin(float(v['A_psi'])*np.pi/180))
    psi_cos.append(np.cos(float(v['A_psi'])*np.pi/180))
    phi_sin.append(np.sin(float(v['A_phi'])*np.pi/180))
    phi_cos.append(np.cos(float(v['A_phi'])*np.pi/180))
    
df_convertedA = pd.DataFrame({'A_psi_sin':  psi_sin, 'A_psi_cos': psi_cos, 'A_phi_sin': phi_cos, 'A_phi_cos': phi_sin})

# state0
psi_sin = []
psi_cos = []
phi_sin = []
phi_cos = []

for i, v in df_gm1cluS0.iterrows():
    psi_sin.append(np.sin(float(v['A_psi'])*np.pi/180))
    psi_cos.append(np.cos(float(v['A_psi'])*np.pi/180))
    phi_sin.append(np.sin(float(v['A_phi'])*np.pi/180))
    phi_cos.append(np.cos(float(v['A_phi'])*np.pi/180))
    
df_convertedS0A = pd.DataFrame({'A_psi_sin':  psi_sin, 'A_psi_cos': psi_cos, 'A_phi_sin': phi_cos, 'A_phi_cos': phi_sin})

# state1
psi_sin = []
psi_cos = []
phi_sin = []
phi_cos = []

for i, v in df_gm1cluS1.iterrows():
    psi_sin.append(np.sin(float(v['A_psi'])*np.pi/180))
    psi_cos.append(np.cos(float(v['A_psi'])*np.pi/180))
    phi_sin.append(np.sin(float(v['A_phi'])*np.pi/180))
    phi_cos.append(np.cos(float(v['A_phi'])*np.pi/180))
    
df_convertedS1A = pd.DataFrame({'A_psi_sin':  psi_sin, 'A_psi_cos': psi_cos, 'A_phi_sin': phi_cos, 'A_phi_cos': phi_sin})

# state2
psi_sin = []
psi_cos = []
phi_sin = []
phi_cos = []

for i, v in df_gm1cluS2.iterrows():
    psi_sin.append(np.sin(float(v['A_psi'])*np.pi/180))
    psi_cos.append(np.cos(float(v['A_psi'])*np.pi/180))
    phi_sin.append(np.sin(float(v['A_phi'])*np.pi/180))
    phi_cos.append(np.cos(float(v['A_phi'])*np.pi/180))
    
df_convertedS2A = pd.DataFrame({'A_psi_sin':  psi_sin, 'A_psi_cos': psi_cos, 'A_phi_sin': phi_cos, 'A_phi_cos': phi_sin})

# state3
psi_sin = []
psi_cos = []
phi_sin = []
phi_cos = []

for i, v in df_gm1cluS3.iterrows():
    psi_sin.append(np.sin(float(v['A_psi'])*np.pi/180))
    psi_cos.append(np.cos(float(v['A_psi'])*np.pi/180))
    phi_sin.append(np.sin(float(v['A_phi'])*np.pi/180))
    phi_cos.append(np.cos(float(v['A_phi'])*np.pi/180))
    
df_convertedS3A = pd.DataFrame({'A_psi_sin':  psi_sin, 'A_psi_cos': psi_cos, 'A_phi_sin': phi_cos, 'A_phi_cos': phi_sin})

# state4
psi_sin = []
psi_cos = []
phi_sin = []
phi_cos = []

for i, v in df_gm1cluS4.iterrows():
    psi_sin.append(np.sin(float(v['A_psi'])*np.pi/180))
    psi_cos.append(np.cos(float(v['A_psi'])*np.pi/180))
    phi_sin.append(np.sin(float(v['A_phi'])*np.pi/180))
    phi_cos.append(np.cos(float(v['A_phi'])*np.pi/180))
    
df_convertedS4A = pd.DataFrame({'A_psi_sin':  psi_sin, 'A_psi_cos': psi_cos, 'A_phi_sin': phi_cos, 'A_phi_cos': phi_sin})

# state5
psi_sin = []
psi_cos = []
phi_sin = []
phi_cos = []

for i, v in df_gm1cluS5.iterrows():
    psi_sin.append(np.sin(float(v['A_psi'])*np.pi/180))
    psi_cos.append(np.cos(float(v['A_psi'])*np.pi/180))
    phi_sin.append(np.sin(float(v['A_phi'])*np.pi/180))
    phi_cos.append(np.cos(float(v['A_phi'])*np.pi/180))
    
df_convertedS5A = pd.DataFrame({'A_psi_sin':  psi_sin, 'A_psi_cos': psi_cos, 'A_phi_sin': phi_cos, 'A_phi_cos': phi_sin})

# state6
psi_sin = []
psi_cos = []
phi_sin = []
phi_cos = []

for i, v in df_gm1cluS6.iterrows():
    psi_sin.append(np.sin(float(v['A_psi'])*np.pi/180))
    psi_cos.append(np.cos(float(v['A_psi'])*np.pi/180))
    phi_sin.append(np.sin(float(v['A_phi'])*np.pi/180))
    phi_cos.append(np.cos(float(v['A_phi'])*np.pi/180))
    
df_convertedS6A = pd.DataFrame({'A_psi_sin':  psi_sin, 'A_psi_cos': psi_cos, 'A_phi_sin': phi_cos, 'A_phi_cos': phi_sin})

# state7
psi_sin = []
psi_cos = []
phi_sin = []
phi_cos = []

for i, v in df_gm1cluS7.iterrows():
    psi_sin.append(np.sin(float(v['A_psi'])*np.pi/180))
    psi_cos.append(np.cos(float(v['A_psi'])*np.pi/180))
    phi_sin.append(np.sin(float(v['A_phi'])*np.pi/180))
    phi_cos.append(np.cos(float(v['A_phi'])*np.pi/180))
    
df_convertedS7A = pd.DataFrame({'A_psi_sin':  psi_sin, 'A_psi_cos': psi_cos, 'A_phi_sin': phi_cos, 'A_phi_cos': phi_sin})

In [5]:
df_convertedA.head(10)

,A_phi_cos,A_phi_sin,A_psi_cos,A_psi_sin
0,-0.397920,0.917420,0.836509,0.547953
1,-0.081192,0.996698,0.853309,0.521406
2,-0.349192,0.937051,0.603212,0.797581
3,-0.102676,0.994715,0.489264,0.872136
4,-0.083314,0.996523,0.680558,0.732694
5,0.033891,0.999426,0.863777,0.503875
6,0.307792,0.951454,0.757995,0.652260
7,0.256332,0.966589,0.580049,0.814581
8,0.271017,0.962575,0.710343,0.703856
9,0.286090,0.958203,0.859104,0.511801


In [6]:
psi_sin = []
psi_cos = []
phi_sin = []
phi_cos = []

for i, v in df_gm1cluB.iterrows():
    psi_sin.append(np.sin(float(v['B_psi'])*np.pi/180))
    psi_cos.append(np.cos(float(v['B_psi'])*np.pi/180))
    phi_sin.append(np.sin(float(v['B_phi'])*np.pi/180))
    phi_cos.append(np.cos(float(v['B_phi'])*np.pi/180))
    
df_convertedB = pd.DataFrame({'B_psi_sin':  psi_sin, 'B_psi_cos': psi_cos, 'B_phi_sin': phi_cos, 'B_phi_cos': phi_sin})

# state0
psi_sin = []
psi_cos = []
phi_sin = []
phi_cos = []

for i, v in df_gm1cluS0.iterrows():
    psi_sin.append(np.sin(float(v['B_psi'])*np.pi/180))
    psi_cos.append(np.cos(float(v['B_psi'])*np.pi/180))
    phi_sin.append(np.sin(float(v['B_phi'])*np.pi/180))
    phi_cos.append(np.cos(float(v['B_phi'])*np.pi/180))
    
df_convertedS0B = pd.DataFrame({'B_psi_sin':  psi_sin, 'B_psi_cos': psi_cos, 'B_phi_sin': phi_cos, 'B_phi_cos': phi_sin})

# state1
psi_sin = []
psi_cos = []
phi_sin = []
phi_cos = []

for i, v in df_gm1cluS1.iterrows():
    psi_sin.append(np.sin(float(v['B_psi'])*np.pi/180))
    psi_cos.append(np.cos(float(v['B_psi'])*np.pi/180))
    phi_sin.append(np.sin(float(v['B_phi'])*np.pi/180))
    phi_cos.append(np.cos(float(v['B_phi'])*np.pi/180))
    
df_convertedS1B = pd.DataFrame({'B_psi_sin':  psi_sin, 'B_psi_cos': psi_cos, 'B_phi_sin': phi_cos, 'B_phi_cos': phi_sin})

# state2
psi_sin = []
psi_cos = []
phi_sin = []
phi_cos = []

for i, v in df_gm1cluS2.iterrows():
    psi_sin.append(np.sin(float(v['B_psi'])*np.pi/180))
    psi_cos.append(np.cos(float(v['B_psi'])*np.pi/180))
    phi_sin.append(np.sin(float(v['B_phi'])*np.pi/180))
    phi_cos.append(np.cos(float(v['B_phi'])*np.pi/180))
    
df_convertedS2B = pd.DataFrame({'B_psi_sin':  psi_sin, 'B_psi_cos': psi_cos, 'B_phi_sin': phi_cos, 'B_phi_cos': phi_sin})

# state3
psi_sin = []
psi_cos = []
phi_sin = []
phi_cos = []

for i, v in df_gm1cluS3.iterrows():
    psi_sin.append(np.sin(float(v['B_psi'])*np.pi/180))
    psi_cos.append(np.cos(float(v['B_psi'])*np.pi/180))
    phi_sin.append(np.sin(float(v['B_phi'])*np.pi/180))
    phi_cos.append(np.cos(float(v['B_phi'])*np.pi/180))
    
df_convertedS3B = pd.DataFrame({'B_psi_sin':  psi_sin, 'B_psi_cos': psi_cos, 'B_phi_sin': phi_cos, 'B_phi_cos': phi_sin})

# state4
psi_sin = []
psi_cos = []
phi_sin = []
phi_cos = []

for i, v in df_gm1cluS4.iterrows():
    psi_sin.append(np.sin(float(v['B_psi'])*np.pi/180))
    psi_cos.append(np.cos(float(v['B_psi'])*np.pi/180))
    phi_sin.append(np.sin(float(v['B_phi'])*np.pi/180))
    phi_cos.append(np.cos(float(v['B_phi'])*np.pi/180))
    
df_convertedS4B = pd.DataFrame({'B_psi_sin':  psi_sin, 'B_psi_cos': psi_cos, 'B_phi_sin': phi_cos, 'B_phi_cos': phi_sin})

# state5
psi_sin = []
psi_cos = []
phi_sin = []
phi_cos = []

for i, v in df_gm1cluS5.iterrows():
    psi_sin.append(np.sin(float(v['B_psi'])*np.pi/180))
    psi_cos.append(np.cos(float(v['B_psi'])*np.pi/180))
    phi_sin.append(np.sin(float(v['B_phi'])*np.pi/180))
    phi_cos.append(np.cos(float(v['B_phi'])*np.pi/180))
    
df_convertedS5B = pd.DataFrame({'B_psi_sin':  psi_sin, 'B_psi_cos': psi_cos, 'B_phi_sin': phi_cos, 'B_phi_cos': phi_sin})

# state6
psi_sin = []
psi_cos = []
phi_sin = []
phi_cos = []

for i, v in df_gm1cluS6.iterrows():
    psi_sin.append(np.sin(float(v['B_psi'])*np.pi/180))
    psi_cos.append(np.cos(float(v['B_psi'])*np.pi/180))
    phi_sin.append(np.sin(float(v['B_phi'])*np.pi/180))
    phi_cos.append(np.cos(float(v['B_phi'])*np.pi/180))
    
df_convertedS6B = pd.DataFrame({'B_psi_sin':  psi_sin, 'B_psi_cos': psi_cos, 'B_phi_sin': phi_cos, 'B_phi_cos': phi_sin})

# state7
psi_sin = []
psi_cos = []
phi_sin = []
phi_cos = []

for i, v in df_gm1cluS7.iterrows():
    psi_sin.append(np.sin(float(v['B_psi'])*np.pi/180))
    psi_cos.append(np.cos(float(v['B_psi'])*np.pi/180))
    phi_sin.append(np.sin(float(v['B_phi'])*np.pi/180))
    phi_cos.append(np.cos(float(v['B_phi'])*np.pi/180))
    
df_convertedS7B = pd.DataFrame({'B_psi_sin':  psi_sin, 'B_psi_cos': psi_cos, 'B_phi_sin': phi_cos, 'B_phi_cos': phi_sin})

In [7]:
psi_sin = []
psi_cos = []
phi_sin = []
phi_cos = []

for i, v in df_gm1cluC.iterrows():
    psi_sin.append(np.sin(float(v['C_psi'])*np.pi/180))
    psi_cos.append(np.cos(float(v['C_psi'])*np.pi/180))
    phi_sin.append(np.sin(float(v['C_phi'])*np.pi/180))
    phi_cos.append(np.cos(float(v['C_phi'])*np.pi/180))
    
df_convertedC = pd.DataFrame({'C_psi_sin':  psi_sin, 'C_psi_cos': psi_cos, 'C_phi_sin': phi_cos, 'C_phi_cos': phi_sin})

# state0
psi_sin = []
psi_cos = []
phi_sin = []
phi_cos = []

for i, v in df_gm1cluS0.iterrows():
    psi_sin.append(np.sin(float(v['C_psi'])*np.pi/180))
    psi_cos.append(np.cos(float(v['C_psi'])*np.pi/180))
    phi_sin.append(np.sin(float(v['C_phi'])*np.pi/180))
    phi_cos.append(np.cos(float(v['C_phi'])*np.pi/180))
    
df_convertedS0C = pd.DataFrame({'C_psi_sin':  psi_sin, 'C_psi_cos': psi_cos, 'C_phi_sin': phi_cos, 'C_phi_cos': phi_sin})

# state1
psi_sin = []
psi_cos = []
phi_sin = []
phi_cos = []

for i, v in df_gm1cluS1.iterrows():
    psi_sin.append(np.sin(float(v['C_psi'])*np.pi/180))
    psi_cos.append(np.cos(float(v['C_psi'])*np.pi/180))
    phi_sin.append(np.sin(float(v['C_phi'])*np.pi/180))
    phi_cos.append(np.cos(float(v['C_phi'])*np.pi/180))
    
df_convertedS1C = pd.DataFrame({'C_psi_sin':  psi_sin, 'C_psi_cos': psi_cos, 'C_phi_sin': phi_cos, 'C_phi_cos': phi_sin})

# state2
psi_sin = []
psi_cos = []
phi_sin = []
phi_cos = []

for i, v in df_gm1cluS2.iterrows():
    psi_sin.append(np.sin(float(v['C_psi'])*np.pi/180))
    psi_cos.append(np.cos(float(v['C_psi'])*np.pi/180))
    phi_sin.append(np.sin(float(v['C_phi'])*np.pi/180))
    phi_cos.append(np.cos(float(v['C_phi'])*np.pi/180))
    
df_convertedS2C = pd.DataFrame({'C_psi_sin':  psi_sin, 'C_psi_cos': psi_cos, 'C_phi_sin': phi_cos, 'C_phi_cos': phi_sin})

# state3
psi_sin = []
psi_cos = []
phi_sin = []
phi_cos = []

for i, v in df_gm1cluS3.iterrows():
    psi_sin.append(np.sin(float(v['C_psi'])*np.pi/180))
    psi_cos.append(np.cos(float(v['C_psi'])*np.pi/180))
    phi_sin.append(np.sin(float(v['C_phi'])*np.pi/180))
    phi_cos.append(np.cos(float(v['C_phi'])*np.pi/180))
    
df_convertedS3C = pd.DataFrame({'C_psi_sin':  psi_sin, 'C_psi_cos': psi_cos, 'C_phi_sin': phi_cos, 'C_phi_cos': phi_sin})

# state4
psi_sin = []
psi_cos = []
phi_sin = []
phi_cos = []

for i, v in df_gm1cluS4.iterrows():
    psi_sin.append(np.sin(float(v['C_psi'])*np.pi/180))
    psi_cos.append(np.cos(float(v['C_psi'])*np.pi/180))
    phi_sin.append(np.sin(float(v['C_phi'])*np.pi/180))
    phi_cos.append(np.cos(float(v['C_phi'])*np.pi/180))
    
df_convertedS4C = pd.DataFrame({'C_psi_sin':  psi_sin, 'C_psi_cos': psi_cos, 'C_phi_sin': phi_cos, 'C_phi_cos': phi_sin})

# state5
psi_sin = []
psi_cos = []
phi_sin = []
phi_cos = []

for i, v in df_gm1cluS5.iterrows():
    psi_sin.append(np.sin(float(v['C_psi'])*np.pi/180))
    psi_cos.append(np.cos(float(v['C_psi'])*np.pi/180))
    phi_sin.append(np.sin(float(v['C_phi'])*np.pi/180))
    phi_cos.append(np.cos(float(v['C_phi'])*np.pi/180))
    
df_convertedS5C = pd.DataFrame({'C_psi_sin':  psi_sin, 'C_psi_cos': psi_cos, 'C_phi_sin': phi_cos, 'C_phi_cos': phi_sin})

# state6
psi_sin = []
psi_cos = []
phi_sin = []
phi_cos = []

for i, v in df_gm1cluS6.iterrows():
    psi_sin.append(np.sin(float(v['C_psi'])*np.pi/180))
    psi_cos.append(np.cos(float(v['C_psi'])*np.pi/180))
    phi_sin.append(np.sin(float(v['C_phi'])*np.pi/180))
    phi_cos.append(np.cos(float(v['C_phi'])*np.pi/180))
    
df_convertedS6C = pd.DataFrame({'C_psi_sin':  psi_sin, 'C_psi_cos': psi_cos, 'C_phi_sin': phi_cos, 'C_phi_cos': phi_sin})

# state7
psi_sin = []
psi_cos = []
phi_sin = []
phi_cos = []

for i, v in df_gm1cluS7.iterrows():
    psi_sin.append(np.sin(float(v['C_psi'])*np.pi/180))
    psi_cos.append(np.cos(float(v['C_psi'])*np.pi/180))
    phi_sin.append(np.sin(float(v['C_phi'])*np.pi/180))
    phi_cos.append(np.cos(float(v['C_phi'])*np.pi/180))
    
df_convertedS7C = pd.DataFrame({'C_psi_sin':  psi_sin, 'C_psi_cos': psi_cos, 'C_phi_sin': phi_cos, 'C_phi_cos': phi_sin})

In [8]:
psi_sin = []
psi_cos = []
phi_sin = []
phi_cos = []

for i, v in df_gm1cluD.iterrows():
    psi_sin.append(np.sin(float(v['D_psi'])*np.pi/180))
    psi_cos.append(np.cos(float(v['D_psi'])*np.pi/180))
    phi_sin.append(np.sin(float(v['D_phi'])*np.pi/180))
    phi_cos.append(np.cos(float(v['D_phi'])*np.pi/180))
    
df_convertedD = pd.DataFrame({'D_psi_sin':  psi_sin, 'D_psi_cos': psi_cos, 'D_phi_sin': phi_cos, 'D_phi_cos': phi_sin})

# state0
psi_sin = []
psi_cos = []
phi_sin = []
phi_cos = []

for i, v in df_gm1cluS0.iterrows():
    psi_sin.append(np.sin(float(v['D_psi'])*np.pi/180))
    psi_cos.append(np.cos(float(v['D_psi'])*np.pi/180))
    phi_sin.append(np.sin(float(v['D_phi'])*np.pi/180))
    phi_cos.append(np.cos(float(v['D_phi'])*np.pi/180))
    
df_convertedS0D = pd.DataFrame({'D_psi_sin':  psi_sin, 'D_psi_cos': psi_cos, 'D_phi_sin': phi_cos, 'D_phi_cos': phi_sin})

# state1
psi_sin = []
psi_cos = []
phi_sin = []
phi_cos = []

for i, v in df_gm1cluS1.iterrows():
    psi_sin.append(np.sin(float(v['D_psi'])*np.pi/180))
    psi_cos.append(np.cos(float(v['D_psi'])*np.pi/180))
    phi_sin.append(np.sin(float(v['D_phi'])*np.pi/180))
    phi_cos.append(np.cos(float(v['D_phi'])*np.pi/180))
    
df_convertedS1D = pd.DataFrame({'D_psi_sin':  psi_sin, 'D_psi_cos': psi_cos, 'D_phi_sin': phi_cos, 'D_phi_cos': phi_sin})

# state2
psi_sin = []
psi_cos = []
phi_sin = []
phi_cos = []

for i, v in df_gm1cluS2.iterrows():
    psi_sin.append(np.sin(float(v['D_psi'])*np.pi/180))
    psi_cos.append(np.cos(float(v['D_psi'])*np.pi/180))
    phi_sin.append(np.sin(float(v['D_phi'])*np.pi/180))
    phi_cos.append(np.cos(float(v['D_phi'])*np.pi/180))
    
df_convertedS2D = pd.DataFrame({'D_psi_sin':  psi_sin, 'D_psi_cos': psi_cos, 'D_phi_sin': phi_cos, 'D_phi_cos': phi_sin})

# state3
psi_sin = []
psi_cos = []
phi_sin = []
phi_cos = []

for i, v in df_gm1cluS3.iterrows():
    psi_sin.append(np.sin(float(v['D_psi'])*np.pi/180))
    psi_cos.append(np.cos(float(v['D_psi'])*np.pi/180))
    phi_sin.append(np.sin(float(v['D_phi'])*np.pi/180))
    phi_cos.append(np.cos(float(v['D_phi'])*np.pi/180))
    
df_convertedS3D = pd.DataFrame({'D_psi_sin':  psi_sin, 'D_psi_cos': psi_cos, 'D_phi_sin': phi_cos, 'D_phi_cos': phi_sin})

# state4
psi_sin = []
psi_cos = []
phi_sin = []
phi_cos = []

for i, v in df_gm1cluS4.iterrows():
    psi_sin.append(np.sin(float(v['D_psi'])*np.pi/180))
    psi_cos.append(np.cos(float(v['D_psi'])*np.pi/180))
    phi_sin.append(np.sin(float(v['D_phi'])*np.pi/180))
    phi_cos.append(np.cos(float(v['D_phi'])*np.pi/180))
    
df_convertedS4D = pd.DataFrame({'D_psi_sin':  psi_sin, 'D_psi_cos': psi_cos, 'D_phi_sin': phi_cos, 'D_phi_cos': phi_sin})

# state5
psi_sin = []
psi_cos = []
phi_sin = []
phi_cos = []

for i, v in df_gm1cluS5.iterrows():
    psi_sin.append(np.sin(float(v['D_psi'])*np.pi/180))
    psi_cos.append(np.cos(float(v['D_psi'])*np.pi/180))
    phi_sin.append(np.sin(float(v['D_phi'])*np.pi/180))
    phi_cos.append(np.cos(float(v['D_phi'])*np.pi/180))
    
df_convertedS5D = pd.DataFrame({'D_psi_sin':  psi_sin, 'D_psi_cos': psi_cos, 'D_phi_sin': phi_cos, 'D_phi_cos': phi_sin})

# state6
psi_sin = []
psi_cos = []
phi_sin = []
phi_cos = []

for i, v in df_gm1cluS6.iterrows():
    psi_sin.append(np.sin(float(v['D_psi'])*np.pi/180))
    psi_cos.append(np.cos(float(v['D_psi'])*np.pi/180))
    phi_sin.append(np.sin(float(v['D_phi'])*np.pi/180))
    phi_cos.append(np.cos(float(v['D_phi'])*np.pi/180))
    
df_convertedS6D = pd.DataFrame({'D_psi_sin':  psi_sin, 'D_psi_cos': psi_cos, 'D_phi_sin': phi_cos, 'D_phi_cos': phi_sin})

# state7
psi_sin = []
psi_cos = []
phi_sin = []
phi_cos = []

for i, v in df_gm1cluS7.iterrows():
    psi_sin.append(np.sin(float(v['D_psi'])*np.pi/180))
    psi_cos.append(np.cos(float(v['D_psi'])*np.pi/180))
    phi_sin.append(np.sin(float(v['D_phi'])*np.pi/180))
    phi_cos.append(np.cos(float(v['D_phi'])*np.pi/180))
    
df_convertedS7D = pd.DataFrame({'D_psi_sin':  psi_sin, 'D_psi_cos': psi_cos, 'D_phi_sin': phi_cos, 'D_phi_cos': phi_sin})

In [9]:
# データフレームの結合
df_gm1clu_convertedABCD = pd.concat([df_convertedA, df_convertedB, df_convertedC, df_convertedD], axis=1)

# state0
df_gm1clu_convertedS0ABCD = pd.concat([df_convertedS0A, df_convertedS0B, df_convertedS0C, df_convertedS0D], axis=1)
# state1
df_gm1clu_convertedS1ABCD = pd.concat([df_convertedS1A, df_convertedS1B, df_convertedS1C, df_convertedS1D], axis=1)
# state2
df_gm1clu_convertedS2ABCD = pd.concat([df_convertedS2A, df_convertedS2B, df_convertedS2C, df_convertedS2D], axis=1)
# state3
df_gm1clu_convertedS3ABCD = pd.concat([df_convertedS3A, df_convertedS3B, df_convertedS3C, df_convertedS3D], axis=1)
# state4
df_gm1clu_convertedS4ABCD = pd.concat([df_convertedS4A, df_convertedS4B, df_convertedS4C, df_convertedS4D], axis=1)
# state5
df_gm1clu_convertedS5ABCD = pd.concat([df_convertedS5A, df_convertedS5B, df_convertedS5C, df_convertedS5D], axis=1)
# state6
df_gm1clu_convertedS6ABCD = pd.concat([df_convertedS6A, df_convertedS6B, df_convertedS6C, df_convertedS6D], axis=1)
# state7
df_gm1clu_convertedS7ABCD = pd.concat([df_convertedS7A, df_convertedS7B, df_convertedS7C, df_convertedS7D], axis=1)

In [10]:
df_gm1clu_convertedABCD.head(10)

,A_phi_cos,A_phi_sin,A_psi_cos,A_psi_sin,B_phi_cos,B_phi_sin,B_psi_cos,B_psi_sin,C_phi_cos,C_phi_sin,C_psi_cos,C_psi_sin,D_phi_cos,D_phi_sin,D_psi_cos,D_psi_sin
0,-0.397920,0.917420,0.836509,0.547953,0.209764,0.977752,0.794858,0.606796,0.230452,0.973084,0.802477,0.596683,-0.090972,0.995853,0.393483,-0.919332
1,-0.081192,0.996698,0.853309,0.521406,0.367605,0.929982,0.919945,0.392048,-0.410321,0.911941,0.655065,0.755572,0.096044,0.995377,0.198420,-0.980117
2,-0.349192,0.937051,0.603212,0.797581,0.203743,0.979024,0.828190,0.560447,0.159562,0.987188,0.675791,0.737093,0.208134,0.978100,0.260348,-0.965515
3,-0.102676,0.994715,0.489264,0.872136,0.187433,0.982277,0.886688,0.462368,-0.460440,0.887691,0.810405,0.585870,0.124318,0.992242,0.121341,-0.992611
4,-0.083314,0.996523,0.680558,0.732694,0.099818,0.995006,0.699596,0.714539,-0.513763,0.857932,0.678434,0.734661,0.016562,0.999863,0.484852,-0.874596
5,0.033891,0.999426,0.863777,0.503875,0.379159,0.925332,0.920540,0.390649,-0.519488,0.854478,0.651602,0.758561,-0.225545,0.974233,0.542267,-0.840206
6,0.307792,0.951454,0.757995,0.652260,0.268316,0.963331,0.881538,0.472114,-0.290632,0.956835,0.627606,0.778531,0.051288,0.998684,0.465835,-0.884871
7,0.256332,0.966589,0.580049,0.814581,0.342458,0.939533,0.785812,0.618466,0.472403,0.881383,0.785461,0.618912,-0.291569,0.956550,0.357796,-0.933800
8,0.271017,0.962575,0.710343,0.703856,0.163734,0.986505,0.795020,0.606584,0.727724,0.685870,0.823569,0.567216,0.133524,0.991046,0.389798,-0.920901
9,0.286090,0.958203,0.859104,0.511801,-0.407722,0.913106,0.746391,0.665508,0.081079,0.996708,0.832633,0.553825,0.054933,0.998490,0.540810,-0.841145


In [ ]:
# KernelPCAをテスト
from sklearn.decomposition import KernelPCA

# カーネル主成分分析を実行
kpca = KernelPCA(n_components=2)
kpca.fit(df_gm1clu_convertedABCD)

In [ ]:
# データを主成分に射影
ktransformed_gm1clu = kpca.transform(df_gm1clu_convertedABCD)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

counts, xedges, yedges, Image = ax.hist2d(ktransformed_gm1clu[:, 0], ktransformed_gm1clu[:, 1], bins=100, norm=matplotlib.colors.LogNorm(), cmap='jet', cmax=2000)